In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier
import joblib

# =============================
# LOAD DATA
# =============================
df = pd.read_csv("medical_dataset_updated.csv")

df['side_effects'] = df['side_effects'].fillna("").astype(str)

# =============================
# AGE RISK BUCKETS (CLINICAL)
# =============================
def age_risk(age):
    if age < 40:
        return 0
    elif age < 65:
        return 1
    else:
        return 2

df['age_risk'] = df['age'].apply(age_risk)

# =============================
# GENDER RISK (NEUTRAL ENCODING)
# =============================
df['gender_risk'] = df['gender'].map({
    'Male': 0,
    'Female': 0,
    'Other': 0
}).fillna(0)

# ⚠️ gender kept neutral to avoid bias
# Used later only for rule escalation, not ML core

# =============================
# MEDICAL-GRADE LABELING
# =============================
def label_decision(text):
    text = text.lower()

    emergency = [
        'difficulty breathing', 'chest pain', 'seizure',
        'loss of consciousness', 'anaphylaxis'
    ]

    see_doctor = [
        'severe', 'persistent vomiting', 'high fever',
        'confusion', 'blurred vision'
    ]

    for k in emergency:
        if k in text:
            return 'Emergency'

    for k in see_doctor:
        return 'See_Doctor'

    return 'Continue'

df['decision'] = df['side_effects'].apply(label_decision)

# =============================
# ENCODERS
# =============================
le_drug = LabelEncoder()
df['drug_enc'] = le_drug.fit_transform(df['drug_name'])

le_cond = LabelEncoder()
df['cond_enc'] = le_cond.fit_transform(df['medical_condition'])

le_target = LabelEncoder()
y = le_target.fit_transform(df['decision'])

# =============================
# TF-IDF (HIGH PRECISION)
# =============================
vectorizer = TfidfVectorizer(
    max_features=1200,
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.9,
    stop_words='english',
    sublinear_tf=True
)

X_text = vectorizer.fit_transform(df['side_effects'])

# =============================
# CORE ML FEATURES ONLY
# =============================
X_struct = df[['drug_enc', 'cond_enc']].values
X_core = np.hstack([X_text.toarray(), X_struct])

# =============================
# SPLIT
# =============================
X_train, X_test, y_train, y_test, age_train, age_test = train_test_split(
    X_core,
    y,
    df['age_risk'],
    test_size=0.2,
    stratify=y,
    random_state=42
)

# =============================
# BASE MODEL
# =============================
base_model = XGBClassifier(
    n_estimators=400,
    max_depth=7,
    learning_rate=0.03,
    subsample=0.85,
    colsample_bytree=0.85,
    objective='multi:softprob',
    num_class=len(le_target.classes_),
    eval_metric='mlogloss',
    random_state=42
)

model = CalibratedClassifierCV(
    base_model,
    method='isotonic',
    cv=3
)

model.fit(X_train, y_train)

# =============================
# EVALUATION
# =============================
preds = model.predict(X_test)

print("\n📊 Medical-Grade Report:\n")
print(classification_report(
    y_test,
    preds,
    target_names=le_target.classes_
))

# =============================
# SAVE
# =============================
joblib.dump(model, 'medical_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
joblib.dump(le_drug, 'drug_encoder.pkl')
joblib.dump(le_cond, 'condition_encoder.pkl')
joblib.dump(le_target, 'decision_encoder.pkl')

print("✅ Scientifically safe model trained successfully")






📊 Medical-Grade Report:

              precision    recall  f1-score   support

   Emergency       1.00      0.98      0.99       297
  See_Doctor       0.98      1.00      0.99       263

    accuracy                           0.99       560
   macro avg       0.99      0.99      0.99       560
weighted avg       0.99      0.99      0.99       560

✅ Scientifically safe model trained successfully
